<a href="https://colab.research.google.com/github/salemsama/nn-zero-to-hero/blob/main/makemore1/makemore_part1_E05_cross_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# E05: look up and use F.cross_entropy instead. You should achieve the same result.
# Can you think of why we'd prefer to use F.cross_entropy instead?

# result:
# achieve the same thing
# I think we'd prefer it becuase its already exist function that is opitmized for this job wich mean it would do more efficently


import torch

words = open("names.txt", 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
words[0]

'emma'

In [4]:
# this is trigram and exercise on bigram
# nerualnetwork version
xs = []
ys = []

for w in words:
  #we add 2 dots because we need a context (learned this idea from the video after this aka makemore 2)
  chs = ['.','.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    xs.append(stoi[ch1]*27 + stoi[ch2])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

print(f"xs: {xs} \nys: {ys}")

#P = (N+1).float()
#P /= P.sum(1, keepdims=True)
#g = torch.Generator().manual_seed(2147483647)

xs: tensor([  0,   5, 148,  ..., 727, 701, 726]) 
ys: tensor([ 5, 13, 13,  ..., 26, 24,  0])


In [5]:
# encodeing xs
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes = 729).float() # 729 = 27 * 27 because we have 2 chars

In [6]:
xenc[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [7]:
xenc.shape # n * 729 where n is number of input aka

torch.Size([228146, 729])

In [8]:
# initilize weights
gen = torch.Generator().manual_seed(2147483647)
w = torch.randn((729,27), generator=gen, requires_grad=True)

In [16]:
for k in range(100):
  # forward pass
  logits = w[xs]     #xenc @ w
  loss = F.cross_entropy(logits,ys, label_smoothing=0.001)
  # backward pass
  w.grad = None # zero grad
  loss.backward()

  #update
  w.data += -10 * w.grad
  print(loss.item())


2.2991600036621094
2.2991139888763428
2.2990684509277344
2.299022674560547
2.2989768981933594
2.298931360244751
2.2988855838775635
2.298840284347534
2.2987945079803467
2.2987494468688965
2.298703670501709
2.298658609390259
2.2986130714416504
2.298567771911621
2.298522710800171
2.2984771728515625
2.298431873321533
2.298386573791504
2.298341989517212
2.2982966899871826
2.2982516288757324
2.298206329345703
2.2981619834899902
2.298116683959961
2.298072099685669
2.2980270385742188
2.2979822158813477
2.2979373931884766
2.2978928089141846
2.2978482246398926
2.2978034019470215
2.2977590560913086
2.2977144718170166
2.2976696491241455
2.2976255416870117
2.2975807189941406
2.2975363731384277
2.297492027282715
2.297447681427002
2.297403335571289
2.2973592281341553
2.2973146438598633
2.2972707748413086
2.2972261905670166
2.297182083129883
2.297138214111328
2.2970941066741943
2.2970502376556396
2.297006130218506
2.296962261199951
2.2969186305999756
2.296874523162842
2.296830654144287
2.2967870235443

In [ ]:
loss.item()

2.9721181392669678

In [18]:
#sampling
for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
    xenc = [ix1*27 + ix2]
    xenc = F.one_hot(torch.tensor(xenc), num_classes=729).float()
    logits = xenc @ w
    counts = logits.exp()
    p = counts / counts.sum(1, keepdims=True)

    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=gen).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break
  print(''.join(out))


amega.
chany.
sufibxqdgkri.
zieqilia.
raanvinegnxiyah.
